In [144]:
from os import listdir, rename
from os.path import isfile, join
from shutil import copyfile

import cv2
import numpy as np

import cPickle as pickle

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from ball_detect_utils import hsv_to_bool_mask
from ball_detect_utils import hsv_to_circular_bool_mask

In [145]:
data = pickle.load(open("data.p", "rb"))

In [159]:
def jaccard(im_mask_pd, im_mask_gt):
    """
    - both input must be bool
    - return between 0 to 1, 1 the best
    """
    assert im_mask_pd.dtype == np.bool
    assert im_mask_gt.dtype == np.bool
    union = float(np.sum(np.logical_or(im_mask_pd, im_mask_gt)))
    interset = float(np.sum(np.logical_and(im_mask_pd, im_mask_gt)))
    if union == 0:
        if interset != 0:
            return -1.0 # more panalty
        else:
            return 0.
    return interset / union

def hsv_to_jaccard(im_hsv, hsv_lows, hsv_highs,
                   im_mask_gt, enable_circular):
    if enable_circular:
        im_mask_pd = hsv_to_circular_bool_mask(im_hsv, 
                                               hsv_lows, hsv_highs)
    else:
        im_mask_pd = hsv_to_bool_mask(im_hsv, hsv_lows, hsv_highs)
    return jaccard(im_mask_pd, im_mask_gt)

def threshold_to_score(data,
                       hsv_lows, hsv_highs,
                       color,
                       enable_circular):
    jacards = []
    
    if color == 'orange':
        for key, d in data.iteritems():
            jacard = hsv_to_jaccard(d['im_hsv'], 
                                    hsv_lows, hsv_highs,
                                    d['im_mask_orange'], enable_circular)
            jacards.append(jacard)
    elif color == 'green':
        for key, d in data.iteritems():
            jacard = hsv_to_jaccard(d['im_hsv'], 
                                    hsv_lows, hsv_highs,
                                    d['im_mask_green'], enable_circular)
            jacards.append(jacard)
    else:
        raise
        
    return float(np.sum(jacards))

In [160]:
orange_hsv_lows = (0, 146, 120)
orange_hsv_highs = (16, 255, 255)
green_hsv_lows = (30, 80, 80)
green_hsv_highs = (50, 255, 255)

In [161]:
threshold_to_score(data,
                   green_hsv_lows, green_hsv_highs,
                   color='green', enable_circular=False)

18.266876805112197

In [170]:
from hyperopt import hp
from hyperopt import fmin, tpe

from hyperopt import fmin, tpe, hp, Trials
from hyperopt.mongoexp import MongoTrials

def objective(args):
    hsv_lows, hsv_highs, color, enable_circular = args
    return -threshold_to_score(data, hsv_lows, hsv_highs,
                               color, enable_circular)

space = ((hp.uniform('h_low', 0, 50), hp.uniform('s_low', 0, 160), hp.uniform('v_low', 0, 160)), 
         (hp.uniform('h_high', 20, 100), hp.uniform('s_high', 254, 255), hp.uniform('v_high', 254, 255)), 
         'green', False)

In [171]:
best = fmin(objective, space, algo=tpe.suggest, max_evals=100)

In [172]:
print best
print objective(hyperopt.space_eval(space, best))

{'v_low': 37.71499753830188, 's_low': 115.21630413498997, 's_high': 254.38179846923157, 'h_high': 86.29366114702698, 'h_low': 36.614939734020396, 'v_high': 254.71606894371214}
-51.3020191444
